http://www.postgresqltutorial.com/postgresql-python/create-tables/

In [1]:
import psycopg2

import os

/home/devopsgong/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
from datetime import datetime

In [3]:
db_host = os.environ.get('AWS_PG_DB_HOST')
db_name = os.environ.get('AWS_PG_DB_NAME')
db_user = os.environ.get('AWS_PG_DB_USER')
password = os.environ.get('AWS_PG_DB_PASS')

db_connection_string = f"dbname='{db_name}' " + \
    f"user='{db_user}' " + \
    f"host='{db_host}' " + \
    f"password='{password}'"

In [4]:
connection = psycopg2.connect(db_connection_string)

In [5]:
cur = connection.cursor()

In [6]:
cur.execute("select * from person;")

rows = cur.fetchall()

print(rows)

[('wen', 'gong', 'wen.gong@gmail.com'), ('anna', 'wang', 'anna.wang@yahoo.com'), ('albert', 'gong', 'al.gong@ncssm.org'), ('anna2', 'wang2', 'anna.wang2@yahoo.com')]


In [19]:
cur.execute("select * from xml_txns;")
rows = cur.fetchall()
print(rows)

[(1, 'Traffic/2019-01-28/1839_Trafficspeed.gz', datetime.datetime(2019, 1, 28, 17, 44, 19), None, 0, 2), (2, 'Traffic/2019-01-28/1839_Trafficspeed.gz', datetime.datetime(2019, 1, 29, 3, 24, 32, 273341), None, 0, 2)]


In [14]:
begin_datetime = datetime.utcnow()
begin_datetime

filename="Traffic/2019-01-28/1839_Trafficspeed.gz"

cur.execute(f"""
    INSERT INTO xml_txns (filename, begin_datetime) 
    VALUES ('{filename}', '{begin_datetime}') 
    ;
""")

connection.commit()

datetime.datetime(2019, 1, 29, 3, 24, 32, 273341)

In [ ]:
begin_datetime = datetime.utcnow()
begin_datetime

filename="Traffic/2019-01-28/1859_Trafficspeed.gz"

cur.execute(f"""
    INSERT INTO xml_txns (filename, begin_datetime) 
    VALUES ('{filename}', '{begin_datetime}') 
    returning id
    ;
""")

In [32]:
connection.commit()

In [23]:
rows = cur.fetchall()
id = rows[0][0]
print(id)

5


In [24]:
def add_txn(connection, filename, begin_datetime):
    time_stamp = datetime.utcnow()
    cur = connection.cursor()
    cur.execute(f"""
        INSERT INTO xml_txns (filename, begin_datetime) 
        VALUES ('{filename}', '{begin_datetime}') 
        returning id
        ;
    """)
    connection.commit()
    rows = cur.fetchall()
    cur.close()
    return rows[0][0]

In [29]:
def upd_txn(connection, id, status, num_locations=0):
    end_datetime = datetime.utcnow()
    cur = connection.cursor()
    cur.execute(f"""
        UPDATE xml_txns 
        SET 
            end_datetime='{end_datetime}' 
            ,status={status}
            ,num_locations={num_locations} 
        WHERE id = {id};
    """)
    connection.commit()
    cur.close()

In [30]:
filename, begin_datetime ="Traffic/2019-01-28/1859_Trafficspeed.gz", datetime.utcnow()

In [31]:
print(add_txn(connection, filename, begin_datetime))

InternalError: current transaction is aborted, commands ignored until end of transaction block


In [33]:
upd_txn(connection, 6, 0, num_locations=100)

In [34]:
cur.execute("select * from xml_txns where id=6;")
rows = cur.fetchall()
print(rows)

[(6, 'Traffic/2019-01-28/1859_Trafficspeed.gz', datetime.datetime(2019, 1, 29, 3, 31, 52, 147585), datetime.datetime(2019, 1, 29, 3, 43, 41, 368948), 100, 0)]


In [6]:
cur.execute("""CREATE TABLE IF NOT EXISTS person (
	first_name text, last_name text, email text,
	PRIMARY KEY (email)
	);""")

In [7]:
connection.commit()

In [11]:
cur.execute("""insert into person (first_name, last_name, email) 
    values('anna2','wang2','anna.wang2@yahoo.com');
""")

In [12]:
connection.commit()

In [16]:
cur.close()

In [17]:
connection.close()